In [5]:
import tensorflow as tf
from train import build_model
from preprocess_data import load_data
from layers import *

In [6]:
batch_size = 32
model = build_model()
train_dataset, test_dataset = load_data(data_path="/home/pbr-student/personal/thesis/test/PedestrianTrajectoryPrediction/final_dataset.pkl", batch_size=batch_size)

(15,)


2024-07-04 10:20:58.734577: W tensorflow/core/framework/dataset.cc:956] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2024-07-04 10:21:28.734952: W tensorflow/core/framework/dataset.cc:956] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
